# TRAINING I3W


# A) Create Envorinment, Vehicles etc

### General Parameter

In [1]:
# Define horizon as a variable to ensure consistent use across notebook (length of one rollout)
HORIZON=500

# name of the experiment
experiment_name = "IntersectionExample"

# scenario class
import flow.scenarios as scenarios
print("Available scenarios:")
print(scenarios.__all__)
scenario_name = "IntersectionTWScenario"

# environment class
import flow.multiagent_envs as flowenvs
print("\nAvailable environments:")
print(flowenvs.__all__)
env_name = "MultiAgentIntersectionEnv"

Available scenarios:
['Scenario', 'BayBridgeScenario', 'BayBridgeTollScenario', 'BottleneckScenario', 'Figure8Scenario', 'SimpleGridScenario', 'HighwayScenario', 'LoopScenario', 'MergeScenario', 'TwoLoopsOneMergingScenario', 'MultiLoopScenario', 'IntersectionScenario', 'IntersectionScenarioTW']

Available environments:
['MultiEnv', 'MultiAgentAccelEnv', 'MultiWaveAttenuationPOEnv', 'MultiAgentIntersectionEnv']


### Net Parameter

In [2]:
from flow.core.params import NetParams
from flow.scenarios.intersection import ADDITIONAL_NET_PARAMS

additionalNetParams = {
            "edge_length": 40,
            "lanes": 1,
            "speed_limit": 30
        }

net_params = NetParams( no_internal_links=False,                  #default: True   !! damit Kreuzungen nicht überspr. werden
                        inflows=None,                             #default: None
                        osm_path=None,                            #default: None
                        netfile=None,                             #default: None
                        additional_params=additionalNetParams     #default: None   !!
                      )

### InitialConfig Parameter

In [3]:
from flow.core.params import InitialConfig

initial_config = InitialConfig( shuffle=True,                            #default: False         !!
                                spacing="custom",                        #default: "uniform"     !!
                                min_gap=10,                              #default: 0
                                perturbation=39.99,                      #default: 0.0            !!        
                                x0=0,                                    #default: 0
                                bunching=0,                              #default: 0
                                lanes_distribution=float("inf"),         #default: float("inf")
                                edges_distribution="all",                #default: "all"
                                additional_params=None )                 #default: None

### SUMO Parameter

In [4]:
from flow.core.params import SumoParams

sumo_params = SumoParams( port = None,                  #default: None
                          sim_step=0.1,                 #default: 0.1
                          emission_path=None,           #default: None
                          lateral_resolution=None,      #default: None
                          no_step_log=True,             #default: True
                          render=False,                 #default: False
                          save_render=False,            #default: False
                          sight_radius=25,              #default: 25
                          show_radius=False,            #default: False
                          pxpm=2,                       #default: 2
                          overtake_right=False,         #default: False    
                          seed=None,                    #default: None
                          restart_instance=False,       #default: False
                          print_warnings=True,          #default: True
                          teleport_time=-1,             #default: -1
                          num_clients=1,                #default: 1
                          sumo_binary=None )            #default: None

### Environment Parameter

In [5]:
from flow.core.params import EnvParams

additionalEnvParams = {
        # maximum acceleration of autonomous vehicles
        "max_accel": 3,
        # maximum deceleration of autonomous vehicles
        "max_decel": 3,
        "target_velocity": 30
    }

env_params = EnvParams( additional_params=additionalEnvParams, #default: None    !!
                        horizon=HORIZON,                       #default: 500     !!
                        warmup_steps=0,                        #default: 0       
                        sims_per_step=1,                       #default: 1
                        evaluate=False )                       #default: False

### Vehicles Parameter

In [6]:
from flow.core.params import VehicleParams

# import vehicles dynamics models
#from flow.controllers import SumoCarFollowingController
from flow.controllers import ContinuousRouter
#from flow.controllers.lane_change_controllers import SumoLaneChangeController
from flow.controllers.lane_change_controllers import StaticLaneChanger
from flow.controllers import RLController
from flow.core.params import SumoLaneChangeParams
from flow.core.params import SumoCarFollowingParams
from random import *

vehicles = VehicleParams()

#### Add RL-Agent controlled vehicles 

In [7]:
# car following parameters, default: None
cf_parameter = SumoCarFollowingParams(
                speed_mode="aggressive")
# lane change parameters, default: None
lc_parameter =  None

vehicles.add( # name of the vehicle
                veh_id = "rl",
              # acceleration controller, default: (SumoCarFollowingController, {})
                acceleration_controller=(RLController, {}),
              # lane_change_controller, default: (SumoLaneChangeController, {})
                lane_change_controller=(StaticLaneChanger,{}),
              # routing controller, default: None
                routing_controller=(ContinuousRouter, {}),
              # initial speed, default: 0
                initial_speed=0,
              # number of vehicles, default: 1 
                num_vehicles=2,
                
                car_following_params=cf_parameter
              # speed mode, default: "right_of_way"
                #speed_mode="aggressive",
              # lane change mode, default: "no_lat_collide"
                #lane_change_mode="aggressive", 
              # car following parameter, default: None
                #sumo_car_following_params=cf_parameter,
              # lane change parameter, default: None
                #sumo_lc_params=lc_parameter
)

### Flow Parameter

In [8]:
# Creating flow_params. Make sure the dictionary keys are as specified. 
flow_params = dict( # name of the experiment
                      exp_tag=experiment_name,
                    # name of the flow environment the experiment is running on
                      env_name=env_name,
                    # name of the scenario class the experiment uses
                      scenario=scenario_name,
                    # simulator that is used by the experiment
                      simulator='traci',
                    # sumo-related parameters (see flow.core.params.SumoParams)
                      sim=sumo_params,
                    # environment related parameters (see flow.core.params.EnvParams)
                      env=env_params,
                    # network-related parameters (see flow.core.params.NetParams and
                    # the scenario's documentation or ADDITIONAL_NET_PARAMS component)
                      net=net_params,
                    # vehicles to be placed in the network at the start of a rollout 
                    # (see flow.core.vehicles.Vehicles)
                      veh=vehicles,
                   # (optional) parameters affecting the positioning of vehicles upon 
                   # initialization/reset (see flow.core.params.InitialConfig)
                      initial=initial_config
                )

# B) Training

In [9]:
import json

import ray
try:
    from ray.rllib.agents.agent import get_agent_class
except ImportError:
    from ray.rllib.agents.registry import get_agent_class
from ray.tune import run_experiments
from ray.tune.registry import register_env

from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder

from ray import tune
from ray.rllib.agents.ppo.ppo_policy_graph import PPOPolicyGraph

In [10]:
# number of parallel workers
N_CPUS = 2
# number of rollouts per training iteration
N_ROLLOUTS = 20

ray.init(redirect_output=True, num_cpus=N_CPUS+1)

Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-02-22_00-55-48_9744/logs.
Waiting for redis server at 127.0.0.1:34548 to respond...
Waiting for redis server at 127.0.0.1:26306 to respond...
Starting the Plasma object store with 6.554658406 GB memory using /dev/shm.

View the web UI at http://localhost:8889/notebooks/ray_ui.ipynb?token=29bcce1bdc62dd4c2b2de9bde51d3a1d039d9d2877165009



{'node_ip_address': '192.168.2.102',
 'object_store_addresses': ['/tmp/ray/session_2019-02-22_00-55-48_9744/sockets/plasma_store'],
 'raylet_socket_names': ['/tmp/ray/session_2019-02-22_00-55-48_9744/sockets/raylet'],
 'redis_address': '192.168.2.102:34548',
 'webui_url': 'http://localhost:8889/notebooks/ray_ui.ipynb?token=29bcce1bdc62dd4c2b2de9bde51d3a1d039d9d2877165009'}

In [11]:
# The algorithm or model to train. This may refer to "
#      "the name of a built-on algorithm (e.g. RLLib's DQN "
#      "or PPO), or a user-defined trainable function or "
#      "class registered in the tune registry.")
alg_run = "PPO"

agent_cls = get_agent_class(alg_run)
config = agent_cls._default_config.copy()
config["num_workers"] = N_CPUS  # number of parallel workers
config["train_batch_size"] = HORIZON * N_ROLLOUTS  # batch size
config["gamma"] = 0.999  # discount rate
config["model"].update({"fcnet_hiddens": [64, 32]})  # size of hidden layers in network
config["use_gae"] = True  # using generalized advantage estimation
config["lambda"] = 0.97  
#config["sgd_minibatch_size"] = min(16 * 1024, config["train_batch_size"])  # stochastic gradient descent
#config["sample_batch_size"] = config["train_batch_size"]/config["num_workers"] # 200 default, trotzdem zu hoch?
config["kl_target"] = 0.02  # target KL divergence
config["num_sgd_iter"] = 10  # number of SGD iterations
config["horizon"] = HORIZON  # rollout horizon

# save the flow params for replay
flow_json = json.dumps(flow_params, cls=FlowParamsEncoder, sort_keys=True,
                       indent=4)  # generating a string version of flow_params
config['env_config']['flow_params'] = flow_json  # adding the flow_params to config dict
config['env_config']['run'] = alg_run

# Call the utility function make_create_env to be able to 
# register the Flow env for this experiment
create_env, gym_name = make_create_env(params=flow_params, version=0)

# Register as rllib env with Gym
register_env(gym_name, create_env)

In [12]:
# multi agent policy mapping
test_env = create_env()
obs_space = test_env.observation_space
act_space = test_env.action_space

def gen_policy():
    return (PPOPolicyGraph, obs_space, act_space, {})

# Setup PG with an ensemble of `num_policies` different policy graphs
policy_graphs = {'rl_0': gen_policy(), 'rl_1': gen_policy()}
    
def policy_mapping_fn(agent_id):
    return agent_id

config.update({
        'multiagent': {
            'policy_graphs': policy_graphs,
            'policy_mapping_fn': tune.function(policy_mapping_fn)
        }
    })

 Starting SUMO on port 41527


18.60796873449424
1.6004638567410308


In [13]:
trials = run_experiments({
    flow_params["exp_tag"]: {
        "run": alg_run,  # RL algorithm to run
        "env": gym_name,  # environment name generated earlier
        "config": {  # configuration params (must match "run" value)
            **config
        },
        "checkpoint_freq": 1,  # number of iterations between checkpoints
        "max_failures": 999,
        "stop": {  # stopping conditions
            "training_iteration": 1000,  # number of iterations to stop after
        },
    },
})

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/3 CPUs, 0/1 GPUs
Memory usage on this node: 5.5/16.4 GB

Created LogSyncer for /home/thorsten/ray_results/IntersectionExample/PPO_MultiAgentIntersectionEnv-v0_0_2019-02-22_00-55-50s2hylffo -> 
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 5.5/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-02-22_00-57-26
  done: false
  episode_len_mean: 452.3333333333333
  episode_reward_max: 150.5330812030182
  episode_reward_mean: 58.40426735411326
  episode_reward_min: -133.99335420565546
  episodes_this_iter: 21
  episodes_total: 21
  experiment_id: e9a0a10898fb45ea8bdfb380c7f1a8bb
  hostname: Gandalf
  info:
    grad_time_ms: 6746.774
    load_time_ms: 195.097
    num_steps_sampled: 10000
    num_st

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-02-22_00-59-42
  done: false
  episode_len_mean: 410.83
  episode_reward_max: 231.32417470785919
  episode_reward_mean: 43.186451504935505
  episode_reward_min: -181.00289670748407
  episodes_this_iter: 26
  episodes_total: 118
  experiment_id: e9a0a10898fb45ea8bdfb380c7f1a8bb
  hostname: Gandalf
  info:
    grad_time_ms: 5356.996
    load_time_ms: 41.405
    num_steps_sampled: 50000
    num_steps_trained: 50000
    rl_0:
      cur_kl_coeff: 0.012500000186264515
      cur_lr: 4.999999873689376e-05
      entropy: 1.4217296838760376
      kl: 0.0008324080263264477
      policy_loss: -0.001443819492124021
      total_loss: 90.03878784179688
      vf_explained_var: 0.1024768203496933
      vf_loss: 90.04022979736328
    rl_1:
      cur_kl_coeff: 0.012500000186264515
      cur_lr: 4.999999873689376e-05
      entropy: 1.4223064184188843
      kl: 0.008418683893978596
      policy_loss: -0.005134430713951588
    

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 6.4/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=9795], 317 s, 9 iter, 90000 ts, 57.6 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-02-22_01-02-23
  done: false
  episode_len_mean: 278.2
  episode_reward_max: 228.1251415289585
  episode_reward_mean: 59.67285583145381
  episode_reward_min: -178.0155407905042
  episodes_this_iter: 35
  episodes_total: 282
  experiment_id: e9a0a10898fb45ea8bdfb380c7f1a8bb
  hostname: Gandalf
  info:
    grad_time_ms: 4935.097
    load_time_ms: 22.135
    num_steps_sampled: 100000
    num_steps_trained: 100000
    rl_0:
      cur_kl_coeff: 0.0003906250058207661
      cur_lr: 4.999999873689376e-05
      entropy: 1.4096665382385254
      kl: 0.0052779545076191425
      policy_loss: -0.0030526025220751762
      total_

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 6.5/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=9795], 442 s, 14 iter, 140000 ts, 34.5 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-02-22_01-04-34
  done: false
  episode_len_mean: 230.33
  episode_reward_max: 210.0741878541329
  episode_reward_mean: 26.773882147799007
  episode_reward_min: -181.22632574289983
  episodes_this_iter: 41
  episodes_total: 482
  experiment_id: e9a0a10898fb45ea8bdfb380c7f1a8bb
  hostname: Gandalf
  info:
    grad_time_ms: 4156.522
    load_time_ms: 3.024
    num_steps_sampled: 150000
    num_steps_trained: 150000
    rl_0:
      cur_kl_coeff: 1.220703143189894e-05
      cur_lr: 4.999999873689376e-05
      entropy: 1.4177895784378052
      kl: 0.001920485869050026
      policy_loss: -0.0008132904767990112
      tot

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 6.0/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=9795], 596 s, 19 iter, 190000 ts, 83 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-02-22_01-07-02
  done: false
  episode_len_mean: 192.6
  episode_reward_max: 217.9487933486108
  episode_reward_mean: 71.53229679893875
  episode_reward_min: -182.99226451535193
  episodes_this_iter: 55
  episodes_total: 720
  experiment_id: e9a0a10898fb45ea8bdfb380c7f1a8bb
  hostname: Gandalf
  info:
    grad_time_ms: 3829.584
    load_time_ms: 2.998
    num_steps_sampled: 200000
    num_steps_trained: 200000
    rl_0:
      cur_kl_coeff: 3.814697322468419e-07
      cur_lr: 4.999999873689376e-05
      entropy: 1.4036473035812378
      kl: 0.002852262929081917
      policy_loss: -0.0009723320254124701
      total_l

...

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 9.3/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=9795], 23010 s, 979 iter, 9790000 ts, 175 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-02-22_07-21-21
  done: false
  episode_len_mean: 89.00877192982456
  episode_reward_max: 219.92691782574195
  episode_reward_mean: 172.35301989271431
  episode_reward_min: 134.57191434726897
  episodes_this_iter: 114
  episodes_total: 104643
  experiment_id: e9a0a10898fb45ea8bdfb380c7f1a8bb
  hostname: Gandalf
  info:
    grad_time_ms: 3148.575
    load_time_ms: 2.532
    num_steps_sampled: 9800000
    num_steps_trained: 9800000
    rl_0:
      cur_kl_coeff: 0.0
      cur_lr: 4.999999873689376e-05
      entropy: -0.6927513480186462
      kl: 0.021913006901741028
      policy_loss: 0.0011805612593889236
      t

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 9.3/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=9795], 23126 s, 984 iter, 9840000 ts, 174 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-02-22_07-23-19
  done: false
  episode_len_mean: 88.72566371681415
  episode_reward_max: 219.15082119139663
  episode_reward_mean: 166.0422927508677
  episode_reward_min: -155.67650538992325
  episodes_this_iter: 113
  episodes_total: 105205
  experiment_id: e9a0a10898fb45ea8bdfb380c7f1a8bb
  hostname: Gandalf
  info:
    grad_time_ms: 3127.128
    load_time_ms: 2.598
    num_steps_sampled: 9850000
    num_steps_trained: 9850000
    rl_0:
      cur_kl_coeff: 0.0
      cur_lr: 4.999999873689376e-05
      entropy: -0.6548426747322083
      kl: 0.010501096956431866
      policy_loss: -0.0025811451487243176
      

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 9.3/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=9795], 23244 s, 989 iter, 9890000 ts, 174 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-02-22_07-25-16
  done: false
  episode_len_mean: 87.79824561403508
  episode_reward_max: 220.51451930363538
  episode_reward_mean: 173.54931023102603
  episode_reward_min: 140.49670728590198
  episodes_this_iter: 114
  episodes_total: 105770
  experiment_id: e9a0a10898fb45ea8bdfb380c7f1a8bb
  hostname: Gandalf
  info:
    grad_time_ms: 3119.179
    load_time_ms: 2.374
    num_steps_sampled: 9900000
    num_steps_trained: 9900000
    rl_0:
      cur_kl_coeff: 0.0
      cur_lr: 4.999999873689376e-05
      entropy: -0.7044300436973572
      kl: 0.0344371497631073
      policy_loss: 0.0049403090961277485
      tot

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 9.4/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=9795], 23362 s, 994 iter, 9940000 ts, 172 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-02-22_07-27-14
  done: false
  episode_len_mean: 88.54867256637168
  episode_reward_max: 218.4291441224203
  episode_reward_mean: 172.25061893527476
  episode_reward_min: 138.9276787990522
  episodes_this_iter: 113
  episodes_total: 106335
  experiment_id: e9a0a10898fb45ea8bdfb380c7f1a8bb
  hostname: Gandalf
  info:
    grad_time_ms: 3138.589
    load_time_ms: 2.409
    num_steps_sampled: 9950000
    num_steps_trained: 9950000
    rl_0:
      cur_kl_coeff: 0.0
      cur_lr: 4.999999873689376e-05
      entropy: -0.6861922144889832
      kl: 0.026977520436048508
      policy_loss: 0.0005824969848617911
      tot

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 9.4/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=9795], 23479 s, 999 iter, 9990000 ts, 172 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-02-22_07-29-11
  done: true
  episode_len_mean: 88.12389380530973
  episode_reward_max: 216.21244233528506
  episode_reward_mean: 170.08159574685854
  episode_reward_min: -151.20701492835047
  episodes_this_iter: 113
  episodes_total: 106896
  experiment_id: e9a0a10898fb45ea8bdfb380c7f1a8bb
  hostname: Gandalf
  info:
    grad_time_ms: 3132.1
    load_time_ms: 2.481
    num_steps_sampled: 10000000
    num_steps_trained: 10000000
    rl_0:
      cur_kl_coeff: 0.0
      cur_lr: 4.999999873689376e-05
      entropy: -0.6723099946975708
      kl: 0.01799936592578888
      policy_loss: -0.005848770495504141
      to